<a href="https://colab.research.google.com/github/LeNovice/Assignments_WILP_Sem1/blob/master/ACI_ownHeur_26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. WADIKAR PINAK SUHAS [2020sc04158]
2. __________________
3. ____________________
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

# PEAS Description:

The selected Network agent is a Goal-based PSA. Following are the PEAS factors which are considered while preparing this agent.

###Performance Measure:
* Minimizing the Total cost of the generated subgraph
* Nodes are connected in the subgraph (no isolated node)
* Acheiving a better signal to noise ratio along with minimizing the delay between sender and receiver

###Environment:
* Uplink/Downlink from connected nodes
* Valid ping between all the nodes
#####**Assumptions:**

*   Fully Observable: The heuristics between nodes are calculated initially and are known to the Agent
*   Deterministic: No stochastic/Random variables involved
*   Static: Heuristics do not change during any action performed by the agent
*   Discrete: State of environment is governed by heuristic/number of connections, both discrete





###Actuators:
* Ping distortion (delay or noise) to emulate the effect of heuristic between two nodes. 
* Counter to measure the number of active connections in any subgraph
* Display input as well as goal node
Hubs, routers, Master controllers to maintain ping between nodes

###Sensors:
* Ping distortion measuring devices, based on the distortion caused by the heuristic.
* Signal to Noise Ratio detectors for noise distortions
* Skew detectors for Delay distortion.
* Keyboard entries


##Informed Search: Recursive Best First Search
* Best First search requires a goal node which we take in as input from the user to calculate the heuristic value for each node, as per problem definition

Q] Design the agent as PSA Agent(Problem Solving Agent) 
Clear Initial data structures to define the graph and variable declarations is expected 
IMPORTANT: Write distinct code block as below

In [33]:
import sys
from heapq import heappush, heappop
import copy

MAX_DEPTH = 0
heuristic_dict = {}
ISOLATED_PENALTY = 10   #26
MAX_HEUR = 999

In [34]:
#Definition of State: Every node in the network corresponds to a State
class State:
    def __init__(self, Name, Graph, Prev_state=None, Action=None, Heap=[], init_node = 0):
        self.name = Name
        self.graph = Graph
        self.prev_state = Prev_state
        self.heap = Heap                    #State heap will be stored to avoid future re - calculations.
        self.AllNodesVisited = 0
        if (init_node == 1):
            self.h = MAX_HEUR
            self.g = 0
            self.depth=0                        #initial state
#            self.action = [].append(self.name)
        else:
            self.h = self.CalcHeur()
            self.g = self.CalcPathCost()     #We can find this from graph
            Prev_state = self.prev_state
            self.depth = Prev_state.depth + 1
#            self.action = self.prev_state.action.append(Action)
        self.f = self.h + self.g
    #Override comparison functions to compare the states based on their f_value
    def __eq__(self, other):
        if (self.f == other.f):
            return 1
        else:
            return 0
    def __ge__(self,other):
        if (self.f >= other.f):
            return 1
        else:
            return 0
    def __le__(self,other):
        if (self.f <= other.f):
            return 1
        else:
            return 0
    def __lt__(self,other):
        if (self.f < other.f):
            return 1
        else:
            return 0
    def __gt__(self,other):
        if (self.f > other.f):
            return 1
        else:
            return 0
    def __ne__(self,other):
        if (self.f != other.f):
            return 1
        else:
            return 0
    #Function to calculate h(n) for this state
    def CalcHeur(self):
        """
        Heuristics of a state is calculated based on how many nodes are isolated. we penalize 26 units on number of non-connected nodes
        This is just large of the maximum edge cost in the graph
        In order to keep the number of edges minimum, we alo need to add a heuristic to penalize the cost of existing connections however we will look at this penalty as pathcost (stepcost/ cost required to do the action) i.e. g(n) and not add it in h(n)
        """
        current_graph = self.graph
        isolated_penalty = 0
        for key in current_graph:
            conn = current_graph[key]
            if(len(conn) == 0):
                isolated_penalty += ISOLATED_PENALTY
        return isolated_penalty
    #Function to calculate g(n) for this state
    def CalcPathCost(self):
        """
        This is the path cost. Everytime the agent adds another edge to the problem, the additional edge will have some penalty. We consider this as the path cost (g(n))to go towards goal node (goal node where all nodes are connected and h(n) = 0)
        Usually this is calculated while traversal but due to the problem defintion, we can calculate this statically
        Since the graph is bidirectional, the edges will be calculated twice. We will divide by 2 for this
        """
        current_graph = self.graph
        connected_cost = 0
        for key in current_graph:
            conn = current_graph[key]
            if(len(conn) != 0):
                for node, edgecost in conn:
                    connected_cost += edgecost
        return connected_cost/2


In [35]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
graph = {
    'A' : [('B',15),('D',17),('C',10),('G',5)],
    'B' : [('A',15),('D',12)],
    'C' : [('A',10),('G',7)],
    'D' : [('A',17),('B',12),('E',2),('F',10),('H',4)],
    'E' : [('D',2)],
    'F' : [('D',10),('H',11)],
    'G' : [('A',5),('C',7),('H',25)],
    'H' : [('D',4),('G',25),('F',11)]
}
graph_init ={
    'A' : [],
    'B' : [],
    'C' : [],
    'D' : [],
    'E' : [],
    'F' : [],
    'G' : [],
    'H' : []
}
#____________________________________________________DFS variables________________________________________________________________
graph_DFS = {
    'A' : ['B','D','C','G'],
    'B' : ['A','D'],
    'C' : ['A','G'],
    'D' : ['A','B','E','F','H'],
    'E' : ['D'],
    'F' : ['D','H'],
    'G' : ['A','C','H'],
    'H' : ['D','G','F']
}

def graph_AddEdge(state, edge):
    """
    edge is expected to be in the form of (dstnodeB, edgecost)
    """
    curr_graph = state.graph
    curr_nodename = state.name
    nodeB = edge[0]
    edgecost = edge[-1]
    for key, conn in curr_graph:
        if (key == curr_nodename):
            for node_name, edgecost in conn:
                if (node_name == nodeB):
                    sys.exit("GRAPH_ADDEDGE: Connection from " + curr_nodename + " to " + nodeB + " already exists!")
    curr_graph[curr_nodename].append(edge)
    return curr_graph


In [36]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

def succ_fn(node, algo):
    """
    #input: Current state of network Agent
    #output: A heap of Action, Heuristic. For example, ["Go(B)", (heuristic)]
    #Transition Model will return sorted list of possible nodes that we can visit from current node.
    The output is in the form of a min-heap. Each element is (possible_successor_node_name, Heuristic)
    """
    heap = []
    curr_node_name = node.name
    curr_graph = node.graph
    #Explore graph to see what the successors of this node are and also get corresponding heuristic
    if (algo == "BFS"):
        succ_list = graph[curr_node_name]
        #Above will give you all possible connections from current node. This is from main graph
        for node_edgecost in succ_list:
#            if (node_edgecost not in curr_graph[curr_node_name]):
            new_node_name = node_edgecost[0]
            edgecost = node_edgecost[1]
            temp_curr_graph = copy.deepcopy(curr_graph)
#                if(node.g == 0):
                    #This is initial node. Expand all successors unconditionally
                    #Add node from curr node to new node
            temp_curr_graph[curr_node_name].append(node_edgecost)
                    #Similarly add node from new node to curr node
            temp_curr_graph[new_node_name].append((curr_node_name,edgecost))
#                print(curr_graph)
#                print("node_edgecost = " + str(node_edgecost))
#                print(temp_curr_graph)
            new_graph = temp_curr_graph
            heappush(heap, State( new_node_name, new_graph, node, "(" + curr_node_name + "->" + new_node_name + ")"))
#                elif(new_node_name != node.prev_state.name):
#                    temp_curr_graph[curr_node_name].append(node_edgecost)
#                    temp_curr_graph[new_node_name].append((curr_node_name,edgecost))
#                    new_graph = temp_curr_graph
#                    heappush(heap, State( new_node_name, new_graph, node, "(" + curr_node_name + "->" + new_node_name + ")"))
        return heap
    elif (algo == "DFS"):
        succ_list = graph_DFS[curr_node_name]
        return sorted(succ_list)
   

In [37]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
def isGoalTest(node_state):
    return (node_state.h ==0)

### 2.	Definition of Algorithm 1 
INFORMED SEARCH - RECURSIVE BEST FIRST SEARCH)

In [38]:
#Code Block : Function for algorithm 1 implementation
expanded_list = []
RECURSION_LIM = 50
REC_COUNT = 0
def RBFS(node_state, f_limit):
    """
    Assumes that you have already calculated te successor functions of node_state before calling this function.
    For every new node encounteed, you need to do following steps:
    new_node_state = Go(new_node)
    new_node_state.heap = succ_fn(new_node_state)
    BestFirst(new_node_state)
    """
    print("expanding " + node_state.name +" with f_limit = " + str(f_limit))

    global MAX_DEPTH
    global MAX_HEUR
    if(isGoalTest(node_state)):
        print("Goal!!")
        node_state.AllNodesVisited=1
        return node_state, node_state.f
    
    if (len(node_state.heap) > 0):
        #This means that this state was previously visited
        succ_heap = node_state.heap
    else:
        succ_heap = succ_fn(node_state, "BFS")

    #DEBUG Start
    display_heap = copy.deepcopy(succ_heap)
    while(len(display_heap)>0):
        i = heappop(display_heap)
        print(i.name + ":" + str(i.f), end = '\t')
        
    #DEBUG End

    global REC_COUNT
    REC_COUNT = REC_COUNT + 1
    if (REC_COUNT>=RECURSION_LIM):
        sys.exit("Too many recursions")

    while (len(succ_heap) > 0):
        best_state = heappop(succ_heap)
        best_f = best_state.f

        if (best_f >= f_limit):             #>= has been added. equal condition has been added to avoid a going back to parent node after a singluar node (like E) has been expanded (because when E is expanded, )
            node_state.heap = succ_heap
            return best_state, best_f

        single_flag=0
        if (len(succ_heap) == 0):
            single_flag = 1
            alternative_f = f_limit
            print("\nbest = " + best_state.name + " : " + str(best_f) + "\tAlternative = NONE" + " : " + str(alternative_f))
        else:
            single_flag = 0
            alternative_state = heappop(succ_heap)
            alternative_f = alternative_state.f
            print("\nbest = " + best_state.name + " : " + str(best_f) + "\tAlternative = " + alternative_state.name + " : " + str(alternative_f))

        result, new_best_f = RBFS(best_state, min(f_limit,alternative_f))
        MAX_DEPTH = max(MAX_DEPTH,result.depth)
        print("\nReturned from " + best_state.name + " to " + node_state.name + " With new f_value value : " + str(new_best_f))

        if (result.AllNodesVisited==0):             #Equivalent to: if result==failure
            #We push Alternative state before best state so that in case both have same value, the alternative case will be picked first (to avoid looping)
            #Also push the alternative value into heap which was popped earlier. This will be your new best value which will be popped at next iteration of while
            if(single_flag == 0):
                #Only add alternative state if it existed in first case.
                #print("Alternative state " + alternative_state.name + " with f_value = " + str(alternative_f) + " pushed back into heap")
                heappush(succ_heap, alternative_state)
            else:
                print("Seems like all successors of " + node_state.name + " have been visited, breaking....")
                #if current node has only one successor, we have visited it via best. so now we must return to parent and not come back to this
                node_state.heap = succ_heap
                break
            #update the best state with the new best f_value and push the best node with the updated f_value back into successor list
            best_state.f = new_best_f
            heappush(succ_heap, best_state)
        elif(result.AllNodesVisited==1):        #Goal node has been achieved in recursions of this path
            node_state.heap = succ_heap
            return result, new_best_f
    #If succ_heap is empty
    node_state.heap = succ_heap
    return node_state, MAX_HEUR             #node_state.f

### 3.	Definition of Algorithm 2 

UNINFORMED SEARCH: DEPTH LIMITEd SEARCH

In [39]:
#Code Block : Function for algorithm 2 implementation
def DepthLimited(node_state, limit):
    """
    Assumes that you have already calculated te successor functions of node_state before calling this function.
    For every new node encounteed, you need to do following steps:
    new_node_state = Go(new_node)
    new_node_state.heap = succ_fn(new_node_state)
    DepthLimited(new_node_state)
    """
    #Check if current node state is Goal node
    if(isGoalTest(node_state)):
        node_state.AllNodesVisited=1
        return node_state
    else:
        #Get all the successor functions for the current state:
        succ_heap = succ_fn(node_state, "DFS")
        while ((len(succ_heap) > 0) and (limit > 0)):
            next_node_name = heappop(succ_heap)
            if (next_node_name not in node_state.visited_list):
                #New Node encountered. Create a state for it
                next_node_state = Go(node_state, next_node_name, 1, "DFS")
                if(next_node_state is None):
                    sys.exit("DFS ALGO: " + next_node_name + " found in visited list")
                else:
                    #Apply BFS recursively on next node
                    ret = DepthLimited(next_node_state, limit-1)
                    if(isinstance(ret, State)):
                        if(ret.AllNodesVisited == 1):
                            return ret
                        else:
                            curr_visited_list = ret.visited_list
                            curr_visited_list.append(node_state.name)
                            node_state.visited_list = curr_visited_list
        return node_state

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question. 

In [40]:
#Code Block : Function & call to get inputs (start/end state)
init_node_name = 'C'#input("Please enter the start node (A,B,C,D,E,F,G,H):").upper();
goal_node_name = 'A'#input("Please enter the goal node upto which the path needs to be printed (A,B,C,D,E,F,G,H):").upper();


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [41]:
#Code Block : Set Initial State (Must handle dynamic inputs)
init_state = State(init_node_name, graph_init, Prev_state=None, Action=[], Heap=[], init_node=1)

In [42]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
MAX_DEPTH = 0
ret, f_val = RBFS(init_state, MAX_HEUR)
if (ret is None):
    print("Could not find a sub graph which connects all nodes")
elif (isinstance(ret, State)):
    if(ret.AllNodesVisited == 1):
        print("Goal Achieved!\nOptimal Sub Graph:\n")
        for key, conn in ret.graph:
            print(key + ":\t" + str(conn))
        print("Max Depth = " +str(MAX_DEPTH))
    else:
        print("Could not find a sub graph which connects all nodes")

expanding C with f_limit = 999
G:67.0	A:70.0	
best = G : 67.0	Alternative = A : 70.0
expanding G with f_limit = 70.0
A:62.0	C:74.0	H:82.0	
best = A : 62.0	Alternative = C : 74.0
expanding A with f_limit = 70.0
B:67.0	G:67.0	D:69.0	C:72.0	
best = B : 67.0	Alternative = G : 67.0
expanding B with f_limit = 67.0
D:69.0	A:82.0	
Returned from B to A With new f_value value : 69.0

best = G : 67.0	Alternative = D : 69.0
expanding G with f_limit = 69.0
A:72.0	C:74.0	H:82.0	
Returned from G to A With new f_value value : 72.0

best = B : 69.0	Alternative = D : 69.0
expanding B with f_limit = 69.0
A:82.0	
Returned from B to A With new f_value value : 82.0

best = D : 69.0	Alternative = C : 72.0
expanding D with f_limit = 70.0
E:61.0	H:63.0	F:69.0	B:71.0	A:86.0	
best = E : 61.0	Alternative = H : 63.0
expanding E with f_limit = 63.0
D:63.0	
Returned from E to D With new f_value value : 63.0

best = H : 63.0	Alternative = E : 63.0
expanding H with f_limit = 63.0
F:64.0	D:67.0	G:88.0	
Returned from H 

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
init_state = State(init_node_name, [init_node_name], Prev_node=None, Action=None, Cost=999, init_node =1)
LIMIT = 1
MAX_LIMIT = 5
ret = DepthLimited(init_state, LIMIT)
#if (ret is None):
#    print("Could not find a sub graph which connects all nodes")
#elif (isinstance(ret, State)):
#    if(ret.AllNodesVisited == 1):
#        print("Goal Achieved!\nOptimal Sub Graph:\n")
#        print(ret.visited_list)
#    else:
#        print("Cutoff achieved\n Following is the incomplete graph")
#        print(ret.visited_list)
while((ret.AllNodesVisited != 1) and (LIMIT<MAX_LIMIT)):
    init_state = State(init_node_name, [init_node_name], Prev_node=None, Action=None, Cost=999)
    print("Attempt with LIMIT = " + str(LIMIT))
    ret = DepthLimited(init_state, LIMIT)

    if (ret is None):
        sys.exit("Something Went Wrong! Could not find a sub graph which connects all nodes")
    elif (isinstance(ret, State)):
        if(ret.AllNodesVisited == 1):
            print("Goal Achieved!\nOptimal Sub Graph:")
            print(ret.visited_list)
            print("LIMIT_Value = " + str(LIMIT))
        else:
            print("Cutoff achieved\n Following is the incomplete graph")
            Final_visited_list = ret.visited_list
            print(Final_visited_list[:-1])
            print("Retrying with larger LIMIT value ....\n")
            LIMIT = LIMIT + 1

### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 1 
curr_max = 0
for key in graph:
    curr_max = max(curr_max, len(list(graph[key])))
branching_factor = curr_max
print("branching_factor = " + str(branching_factor))
print("MAX_DEPTH of the fully connected sub graph: " + str(MAX_DEPTH))
print("Time Complexity of Recursive Best First Search = Big-Oh(b^l): " + str(branching_factor ** MAX_DEPTH))
print("Space Complexity of Recursive Best First Search = Big-Oh(b*l): " + str(branching_factor*MAX_DEPTH))

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2
curr_max = 0
for key in graph_DFS:
    curr_max = max(curr_max, len(list(graph_DFS[key])))
branching_factor = curr_max
print("branching_factor = " + str(branching_factor))
print("LIMIT at which fully connected sub graph was achieved: " + str(LIMIT))
print("Time Complexity of Depth Limited Search (at limit LIMIT) = Big-O(b^l)", branching_factor**LIMIT)
print("Space Complexity of Depth Limited Search (at limit LIMIT) = Big-O(b*l)", branching_factor*LIMIT)

### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : Apart from time and space complexity, in terms of optimal and completeness, recursive Best First is better that Depth Limited Search.

###|Algorithm| Optimal | Completeness|

|BFS| Yes | Yes |

|DFS| No | No |

##**For Start Node: C and Stop node: A**
###**Optimal connected Sub Graph of Recursive Best First Search:**

['C', 'A', 'G', 'H', 'D', 'E', 'D', 'F', 'D', 'B']

###**Optimal connected Sub Graph of Depth Limited Search:**

['C', 'A', 'B', 'D', 'E', 'D', 'F', 'D', 'H', 'D', 'B', 'A', 'G']